# R3GAN

By [Derrick Schultz](https://twitter.com/dvsch)

No promises...


## Setup

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b5be30c8-70f0-d32b-f7df-03a1e4cf5de5)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdown timm ftfy ninja imageio-ffmpeg opensimplex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/r3gan"):
    %cd "/content/drive/MyDrive/r3gan/R3GAN/"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir r3gan
    %cd r3gan
    !git clone https://github.com/brownvc/R3GAN/
    %cd R3GAN
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !wget https://huggingface.co/brownvc/R3GAN-FFHQ-256x256/resolve/main/network-snapshot-final.pkl -O ./pretrained/ffhq-256.pkl
else:
    !git clone https://github.com/brownvc/R3GAN/
    %cd R3GAN
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !wget https://huggingface.co/brownvc/R3GAN-FFHQ-256x256/resolve/main/network-snapshot-final.pkl -O ./pretrained/ffhq-256.pkl
    %cd ../

/content/drive/MyDrive/r3gan/R3GAN


This cell will update to the latest repo. Git and Drive/Colab don’t play as nicely as I’d like so 🤞. The other option is to delete your folder in Drive (after saving out `/results` and `/datasets`!) and running the script above to replace the entire folder.

In [ ]:
# %cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git pull
# !git stash
# !git checkout origin/main -- train.py gen_images.py gen_video.py README.md training/training_loop.py

## Convert/Create Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!python dataset_tool.py --source=/content/tmp/drawn-gems-1024 --dest=./datasets/drawn-gems-1024.zip

## Training


Working Notes:


In [ ]:
!python train.py --help

In [ ]:
!python train.py --outdir=./training-runs --data=/content/drive/MyDrive/stylegan_xl/data/drawn-gems-256.zip \
--gpus=1 --batch=32 --g-batch-gpu=32 --d-batch-gpu=32 --snap=1 --mirror=1 --preset='FFHQ-256' --metrics=None \
--resume=/content/drive/MyDrive/r3gan/R3GAN/training-runs/00002-drawn-gems-256-gpus1-batch32/network-snapshot-000143870.pkl

## Image Generation

In [ ]:
!python gen_images.py --help

In [ ]:
!python gen_image_debug.py --outdir=/content/drive/MyDrive/r3gan/R3GAN/generated --seeds=2-52 \
    --network=/content/drive/MyDrive/r3gan/R3GAN/training-runs/00032-carpet256_unzipped-gpus1-batch8/network-snapshot-000000034.pkl

## Video Generation
Not yet!


In [ ]:
#script to generate looping vids
import os
import numpy as np
import torch
import PIL.Image
import dnnlib
import legacy
from tqdm import tqdm

network_pkl = "/content/drive/MyDrive/r3gan/R3GAN/training-runs/00032-carpet256_unzipped-gpus1-batch8/network-snapshot-000000035.pkl"
outdir = "/content/drive/MyDrive/r3gan/R3GAN/generated/vid16"
num_frames = 900
radius = 10.0

os.makedirs(outdir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)["G_ema"].to(device)

print("G.z_dim =", G.z_dim)
z_dim = G.z_dim

# Create circular latent path
seed = 53241  # Change this to try different looks
rng = np.random.RandomState(seed)
z_base = rng.randn(1, z_dim)
z_dir1 = rng.randn(1, z_dim)
z_dir2 = rng.randn(1, z_dim)

z_dir1 /= np.linalg.norm(z_dir1)
z_dir2 -= z_dir1 * np.dot(z_dir1, z_dir2.T)  # orthogonalize
z_dir2 /= np.linalg.norm(z_dir2)

for frame in tqdm(range(num_frames), desc="Generating frames"):
    theta = 2 * np.pi * frame / num_frames
    z = z_base + radius * (np.cos(theta) * z_dir1 + np.sin(theta) * z_dir2)
    z_tensor = torch.from_numpy(z).to(device).to(torch.float32)

    label = torch.zeros([1, G.c_dim], device=device)
    img = G(z_tensor, label)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f"{outdir}/frame_{frame:03d}.png")


G.z_dim = 64


Generating frames: 100%|██████████| 900/900 [01:02<00:00, 14.42it/s]


In [ ]:
!ffmpeg -framerate 30 -i /content/drive/MyDrive/r3gan/R3GAN/generated/vid15/frame_%03d.png -c:v libx264 -pix_fmt yuv420p /content/drive/MyDrive/r3gan/R3GAN/generated/latent_loop/loop15.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
#!python gen_video.py --help

In [ ]:
#!python gen_video.py --output=/content/lerp.mp4 --trunc=1 --seeds=100-124 --grid=1x1 --w-frames=72 \
    --network=/content/drive/MyDrive/colab-sg3/stylegan3/results/00014-stylegan3-r-drawn-gems-1024-gpus1-batch32-gamma10/network-snapshot-000104.pkl

In [ ]:
!zip -r /content/out.zip /content/out


	zip warning: name not matched: /content/out

zip error: Nothing to do! (try: zip -r /content/out.zip . -i /content/out)


In [ ]:
# 🚀 Batch Generate 30-Second Latent Loop Videos — All Videos in One Folder

import os
import numpy as np
import torch
import PIL.Image
import dnnlib
import legacy
import subprocess

# ---- CONFIGURATION ----
network_pkl = "/content/drive/MyDrive/r3gan/R3GAN/training-runs/00032-carpet256_unzipped-gpus1-batch8/network-snapshot-000000035.pkl"
outdir = "/content/drive/MyDrive/r3gan/R3GAN/generated/loops"
seeds = [1000, 2000, 3000, 4000, 5000, 6000, 236471, 48592, 125364, 3289, 12673, 1, 59, 35103, 341935, 385294, 4472, 10000, 8000, 9342, 47210, 4278, 4672,
         39624, 7239, 467, 3945, 5284, 60273, 47642, 5626, 57371, 5091742, 34018, 47581, 60264, 64, 59276, 67436, 5630, 5711, 2056, 8582, 58351, 34572, 46,
         8867, 352, 8658, 456, 235, 7457, 2578, 7028, 67264, 69702, 3346, 9572, 5727, 6242, 436, 754, 2, 7, 67, 5764, 23452, 5675, 867, 304, 56658, 9874,
         5463, 35868, 3404, 3419, 459, 6784, 54057, 23478, 45346, 32842, 31878, 674, 58678, 3241, 6784, 67987, 76234, 39241, 4859, 6509, 741770, 34572, 54763,
         12419, 6381, 327491, 4603, 213618, 453491, 3289, 65401, 45728, 214436, 2347456, 56891, 978152469, 1640753622, 209005190, 768400372, 1842041602, 747234424, 222929595, 702832491, 1823931250, 1592649734, 1322570461, 1344741592, 153837313, 1017344335, 140819334, 1381701566, 1621411093, 726816702, 2049493708, 47029113, 939418723, 1345842041, 1422515812, 347635342, 1295513942, 236689735, 2102839257, 1662506407, 1699332921, 778682808, 1809148085, 579474301, 2006541608, 2110904428, 1875044436, 1215831636, 1411440862, 1526088361, 1205466405, 1803132320, 288107249, 690800585, 1233209912, 335038309, 1809788257, 219310004, 725375701, 2067318996, 1547590592, 1025898503, 1063718993, 1061629332, 1724647574, 1291755517, 1403659180, 1250849341, 1963235692, 1336553024, 1138501385, 313506228, 1516635082, 2045108185, 1286480049, 394925646, 1170623569, 1427171627, 2076931801, 145971663, 1326226850, 761591178, 1953076429, 1643976765, 1231456955, 703931825, 658070847, 681983622, 847307201, 1856892427, 1829386077, 1481597334, 1085905271, 845356707, 1168360872, 1068962125, 512235358, 2091177568, 1604878466, 108722157, 876809639, 1324976078, 1501350026, 1435904592]

# ---- REMOVE DUPLICATES + VALIDATE SEEDS ----
seeds = sorted(set(seeds))
max_seed_value = 2**31 - 1
for seed in seeds:
    if not (0 <= seed <= max_seed_value):
        raise ValueError(f"Seed {seed} is out of valid range (0 to {max_seed_value})")

num_frames = 900  # 30 seconds at 30 FPS
fps = 30
z_dim = 64  # z_dim must match your model
radius = 10.0

device = torch.device("cuda")
os.makedirs(outdir, exist_ok=True)

# ---- LOAD NETWORK ----
print(f"Loading network from {network_pkl}...")
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)["G_ema"].to(device)

# ---- GENERATE FOR EACH SEED ----
for seed in seeds:
    print(f"\n🔵 Generating latent loop for seed {seed}...")
    frames_dir = os.path.join(outdir, f"temp_frames_seed_{seed}")
    os.makedirs(frames_dir, exist_ok=True)

    rng = np.random.RandomState(seed)
    z_base = rng.randn(1, z_dim)
    z_dir1 = rng.randn(1, z_dim)
    z_dir2 = rng.randn(1, z_dim)
    z_dir1 /= np.linalg.norm(z_dir1)
    z_dir2 -= z_dir1 * np.dot(z_dir1, z_dir2.T)
    z_dir2 /= np.linalg.norm(z_dir2)

    label = torch.zeros([1, G.c_dim], device=device)

    for frame_idx in range(num_frames):
        theta = 2 * np.pi * frame_idx / num_frames
        z = z_base + radius * (np.cos(theta) * z_dir1 + np.sin(theta) * z_dir2)
        z_tensor = torch.from_numpy(z).to(device).to(torch.float32)

        img = G(z_tensor, label)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        img_np = img[0].cpu().numpy()

        frame_path = os.path.join(frames_dir, f"frame_{frame_idx:04d}.png")
        PIL.Image.fromarray(img_np, 'RGB').save(frame_path)

    print(f"✅ Finished frames for seed {seed}.")

    # ---- CREATE VIDEO FROM FRAMES ----
    video_path = os.path.join(outdir, f"latent_loop_seed_{seed}.mp4")
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-r', str(fps),
        '-i', os.path.join(frames_dir, 'frame_%04d.png'),
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        video_path
    ]
    print(f"🎥 Creating video for seed {seed}...")
    subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # ---- CLEAN UP FRAMES ----
    for frame_file in os.listdir(frames_dir):
        os.remove(os.path.join(frames_dir, frame_file))
    os.rmdir(frames_dir)

    print(f"🎬 Saved and cleaned up: {video_path}")

print("\n✨ All looping videos generated into one folder!")


Loading network from /content/drive/MyDrive/r3gan/R3GAN/training-runs/00032-carpet256_unzipped-gpus1-batch8/network-snapshot-000000035.pkl...

🔵 Generating latent loop for seed 1...
✅ Finished frames for seed 1.
🎥 Creating video for seed 1...
🎬 Saved and cleaned up: /content/drive/MyDrive/r3gan/R3GAN/generated/loops/latent_loop_seed_1.mp4

🔵 Generating latent loop for seed 2...
✅ Finished frames for seed 2.
🎥 Creating video for seed 2...
🎬 Saved and cleaned up: /content/drive/MyDrive/r3gan/R3GAN/generated/loops/latent_loop_seed_2.mp4

🔵 Generating latent loop for seed 7...
✅ Finished frames for seed 7.
🎥 Creating video for seed 7...
🎬 Saved and cleaned up: /content/drive/MyDrive/r3gan/R3GAN/generated/loops/latent_loop_seed_7.mp4

🔵 Generating latent loop for seed 46...
✅ Finished frames for seed 46.
🎥 Creating video for seed 46...
🎬 Saved and cleaned up: /content/drive/MyDrive/r3gan/R3GAN/generated/loops/latent_loop_seed_46.mp4

🔵 Generating latent loop for seed 59...
✅ Finished frames

KeyboardInterrupt: 

In [ ]:
import os
import cv2  # OpenCV for reading frames
from pathlib import Path
from tqdm import tqdm  # Progress bar

# ---- CONFIGURATION ----
input_folder = "/content/drive/MyDrive/r3gan/R3GAN/generated/loops"  # Change this to where your videos/GIFs are
output_folder = "/content/drive/MyDrive/r3gan/R3GAN/generated/first_frames"  # Where first frames will be saved
allowed_extensions = ['.mp4', '.gif']  # File types to process

os.makedirs(output_folder, exist_ok=True)

# ---- PROCESS EACH VIDEO/GIF ----
input_folder = Path(input_folder)
video_files = [f for f in input_folder.rglob('*') if f.suffix.lower() in allowed_extensions]

print(f"Found {len(video_files)} videos/gifs.")

for vid_path in tqdm(video_files, desc="Extracting first frames"):
    cap = cv2.VideoCapture(str(vid_path))
    success, frame = cap.read()
    if success:
        # Save frame as PNG
        base_name = vid_path.stem  # filename without extension
        out_path = os.path.join(output_folder, f"{base_name}_firstframe.png")
        cv2.imwrite(out_path, frame)
    cap.release()

print(f"✅ Done! First frames saved to: {output_folder}")


Found 109 videos/gifs.


Extracting first frames: 100%|██████████| 109/109 [00:01<00:00, 55.04it/s]

✅ Done! First frames saved to: /content/drive/MyDrive/r3gan/R3GAN/generated/first_frames


In [ ]:
import os
import glob
import cv2
import PIL.Image

# ---- CONFIGURATION ----
video_folder = "/content/drive/MyDrive/r3gan/R3GAN/generated/loops"  # Where your videos/gifs are
output_folder = "/content/drive/MyDrive/r3gan/R3GAN/first_frames"     # Where extracted frames will go

os.makedirs(output_folder, exist_ok=True)

# ---- FIND ALL VIDEOS ----
video_files = glob.glob(os.path.join(video_folder, "**/*.mp4"), recursive=True)
video_files += glob.glob(os.path.join(video_folder, "**/*.gif"), recursive=True)

print(f"Found {len(video_files)} videos.")

# ---- EXTRACT FIRST FRAME ----
for vid_path in video_files:
    # Open video or gif
    cap = cv2.VideoCapture(vid_path)
    success, frame = cap.read()

    if success:
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Save as PNG
        vid_name = os.path.splitext(os.path.basename(vid_path))[0]
        frame_path = os.path.join(output_folder, f"{vid_name}_firstframe.png")
        PIL.Image.fromarray(frame).save(frame_path)
        print(f"✅ Saved first frame for {vid_name}")
    else:
        print(f"⚠️ Failed to read {vid_path}")

    cap.release()

print("\n🎉 Done extracting first frames!")


Found 109 videos.
✅ Saved first frame for latent_loop_seed_1
✅ Saved first frame for latent_loop_seed_2
✅ Saved first frame for latent_loop_seed_7
✅ Saved first frame for latent_loop_seed_46
✅ Saved first frame for latent_loop_seed_59
✅ Saved first frame for latent_loop_seed_64
✅ Saved first frame for latent_loop_seed_67
✅ Saved first frame for latent_loop_seed_235
✅ Saved first frame for latent_loop_seed_304
✅ Saved first frame for latent_loop_seed_352
✅ Saved first frame for latent_loop_seed_436
✅ Saved first frame for latent_loop_seed_456
✅ Saved first frame for latent_loop_seed_459
✅ Saved first frame for latent_loop_seed_467
✅ Saved first frame for latent_loop_seed_674
✅ Saved first frame for latent_loop_seed_754
✅ Saved first frame for latent_loop_seed_867
✅ Saved first frame for latent_loop_seed_1000
✅ Saved first frame for latent_loop_seed_2000
✅ Saved first frame for latent_loop_seed_2056
✅ Saved first frame for latent_loop_seed_2578
✅ Saved first frame for latent_loop_seed_30